In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_csv('KWSerpAnalysis 05 - Authority Labs Scrape 2019-09-12.csv')
kw = pd.read_csv('mkt funnel keywords for model.csv')
print(list(kw))

df = df[df['query'].isin(kw['query'])].reset_index(drop=True)

df = df[['query', 'desc']]

df.sample(3)

## Create API Connection for Sentiment Retrieval
- https://cloud.google.com/natural-language/docs/basics#sentiment_analysis
- https://cloud.google.com/natural-language/docs/analyzing-sentiment
- https://cloud.google.com/natural-language/docs/reference/rest/v1/Sentiment

In [ ]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    'credential path.json')

client = language_v1.LanguageServiceClient(credentials=credentials)

def analyze_sentiment(text_content):
    
    document = {"content": text_content,
                "type": enums.Document.Type.PLAIN_TEXT,
                "language": "en"}
    
    response = client.analyze_sentiment(document=document,
                                        encoding_type=enums.EncodingType.UTF8)

    sentiment = response.document_sentiment

    return sentiment.score, sentiment.magnitude

In [ ]:
df['score'] = None
df['magnitude'] = None

## Execute API and Store Results

In [ ]:
for i in range(len(df)):
    print('Item ', i)
    text = df.iloc[i]['desc']
    if type(text) == str:
        sentiment_score, sentiment_magnitude = analyze_sentiment(text)
        df.loc[i, 'score'] = sentiment_score
        print('Sentiment score: ', df.loc[i]['score'])
        df.loc[i, 'magnitude'] = sentiment_magnitude
    else:
        df.loc[i, 'score'] = 0
        df.loc[i, 'magnitude'] = 0

In [ ]:
df.sample(3)

In [ ]:
df.to_csv('example words with sentiments.csv', index=False)